### Szablon funkcji

In [ ]:
import numpy as np
import pytest
from sklearn.linear_model import Ridge

class RidgeRegr:
    def __init__(self, alpha = 0.0):
        self.alpha = alpha

    def fit(self, X, Y):
        # wejscie:
        #  X = np.array, shape = (n, m)
        #  Y = np.array, shape = (n)
        # Znajduje theta (w przyblizeniu) minimalizujace kwadratowa funkcje kosztu L uzywajac metody iteracyjnej.
        n, m = X.shape
        self.theta = np.zeros((m+1))
        return self
    
    def predict(self, X):
        # wejscie
        #  X = np.array, shape = (k, m)
        # zwraca
        #  Y = wektor(f(X_1), ..., f(X_k))
        k, m = X.shape
        return np.zeros((k))


def test_RidgeRegressionInOneDim():
    X = np.array([1,3,2,5]).reshape((4,1))
    Y = np.array([2,5, 3, 8])
    X_test = np.array([1,2,10]).reshape((3,1))
    alpha = 0.3
    expected = Ridge(alpha).fit(X, Y).predict(X_test)
    actual = RidgeRegr(alpha).fit(X, Y).predict(X_test)
    assert list(actual) == pytest.approx(list(expected), rel=1e-5)

def test_RidgeRegressionInThreeDim():
    X = np.array([1,2,3,5,4,5,4,3,3,3,2,5]).reshape((4,3))
    Y = np.array([2,5, 3, 8])
    X_test = np.array([1,0,0, 0,1,0, 0,0,1, 2,5,7, -2,0,3]).reshape((5,3))
    alpha = 0.4
    expected = Ridge(alpha).fit(X, Y).predict(X_test)
    actual = RidgeRegr(alpha).fit(X, Y).predict(X_test)
    assert list(actual) == pytest.approx(list(expected), rel=1e-3)

## Implementacja Ridge

In [4]:
import numpy as np
import pytest
from sklearn.linear_model import Ridge

class RidgeRegr:
    # Musimy przypisać atrybuty do obiektu, alpha, c, tol, max_iter
    def __init__(self, alpha = 0.0, c=0.001, max_iter = 50000, tol = 1e-8):
        self.alpha = alpha
        self.c = c
        self.max_iter = max_iter
        self.tol = tol

    def fit(self, X, Y):
        # wejscie:
        #  X = np.array, shape = (n, m)
        #  Y = np.array, shape = (n)
        # Znajduje theta (w przyblizeniu) minimalizujace kwadratowa funkcje kosztu L uzywajac metody iteracyjnej.
        n, m = X.shape

        # Do X dodajemy kolumnę wyrazów wolnych
        X_z_1 = np.column_stack([np.ones(n),X])

        # Wybieramy jakieś początkowe thety
        self.theta = np.zeros((m+1))
        
    
        # Optymalizujemy thety
        for iteracja in range(self.max_iter):
            y_szacowane = X_z_1 @ self.theta

            # Liczymy gradient
            gradient = np.zeros(m+1) # m+1 współczynników (theta_0, ..., theta_m)

            # Pochodna po theta_0:  suma{i=1}{n} (f(x_i)-yi) 
            gradient[0] = sum(y_szacowane-Y)

            # Pochodne dla theta_1, ..., theta_m. Dla theta_j  suma{i=1}{n} ( xij * [f(x_i)-yi] ) + alfa*theta_j
            # j jest ustalone dla danego theta, zmienia się i
            for j in range(1, m+1):
                gradient[j] = np.sum( X_z_1[:, j] * (y_szacowane-Y) ) + self.alpha*self.theta[j]

            # Aktualizujemy parametry
            nowe_theta = self.theta - self.c * gradient

            # Sprawdzamy, czy odległość euklidesowa jest mniejsza od zadanej tolerancji, tzn. czy theta się zmienia w znaczącym stopniu
            if np.linalg.norm(nowe_theta - self.theta) < self.tol:
                print(iteracja)
                break

            # Ustawiamy nowe, lepsze theta
            self.theta = nowe_theta

        return self

    def predict(self, X):
        # wejscie
        #  X = np.array, shape = (k, m)
        # zwraca
        #  Y = wektor(f(X_1), ..., f(X_k))
        k, m = X.shape
        X_z_1 = np.column_stack([np.ones(k),X])
        # Wyznaczamy wektor Y na podstawie dopasowanego wektora theta
        return X_z_1 @ self.theta


def test_RidgeRegressionInOneDim():
    X = np.array([1,3,2,5]).reshape((4,1))
    Y = np.array([2,5, 3, 8])
    X_test = np.array([1,2,10]).reshape((3,1))
    alpha = 0.3
    expected = Ridge(alpha).fit(X, Y).predict(X_test)
    actual = RidgeRegr(alpha, c=0.01, max_iter=20000).fit(X, Y).predict(X_test)
    print(expected)
    print(actual)
    assert list(actual) == pytest.approx(list(expected), rel=1e-5)
    
def test_RidgeRegressionInThreeDim():
    X = np.array([1,2,3,5,4,5,4,3,3,3,2,5]).reshape((4,3))
    Y = np.array([2,5, 3, 8])
    X_test = np.array([1,0,0, 0,1,0, 0,0,1, 2,5,7, -2,0,3]).reshape((5,3))
    alpha = 0.4
    expected = Ridge(alpha).fit(X, Y).predict(X_test)
    actual = RidgeRegr(alpha, c=0.01, max_iter=80000).fit(X, Y).predict(X_test)
    print(expected)
    print(actual)
    assert list(actual) == pytest.approx(list(expected), rel=1e-3)

test_RidgeRegressionInOneDim()
# Dla początkowego wektora theta (1,1,...,1)
# Dla c=0.01 wystarczy 1556 iteracji, dla c=0.001 potrzeba 12913 iteracji

print("----------")

test_RidgeRegressionInThreeDim() 
# Dla c=0.01 wystarczy 7662 iteracji, dla c=0.001 potrzeba 61186 iteracji

# Nie wyrzuciło wyjątku, zatem funkcja RidgeRegr() poprawnie przeszła testy

1177
[ 1.88950276  3.38121547 15.31491713]
[ 1.88950356  3.38121595 15.31491504]
----------
6177
[ 0.54685378 -1.76188321  1.58691716  5.15527388  3.66704391]
[ 0.54684662 -1.76188799  1.58691183  5.15528139  3.66704223]
